In [ ]:
#| default_exp osmolyteconstraints

In [ ]:
#| include: false
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

In [ ]:
def constrainosmolytelevels(model, osmolytelevels):
    closedphaseos = model.osmolarities[0]/model.getvolumes(perguardcell = False)[0]

    totallevel = 0
    for osmolytelevel in osmolytelevels.values():
        totallevel += osmolytelevel[0][1]
    scaling_closed = closedphaseos/totallevel

    openphaseos = model.osmolarities[1]/model.getvolumes(perguardcell = False)[1]

    totallevel = 0
    for osmolytelevel in osmolytelevels.values():
        totallevel += osmolytelevel[1][1]
    scaling_open = openphaseos/totallevel

    print(scaling_closed, scaling_open)
    
    osmolytelevel_scaled = copy.deepcopy(osmolytelevels)

    for osmolytelevel in osmolytelevel_scaled.values():
        osmolytelevel[0][1] = osmolytelevel[0][1]*scaling_closed
        osmolytelevel[1][1] = osmolytelevel[1][1]*scaling_open
        osmolytelevel[0][1] = osmolytelevel[1][1] # IMPORTANT LINE ignores scaling closed

    otherlist = []
    skipother = ["aMAL", "STARCH"]

    for reaction in model.model.reactions:
        if "gc_Linker_1" in reaction.id:
            osmolytename = "_".join(reaction.id.split("_")[:-4])
            if osmolytename in skipother:
                continue
            other = True
            if osmolytename in osmolytelevel_scaled:
                other = False
            if other == True:
                otherlist.append(osmolytename)
    for osmolyte, concentration in osmolytelevel_scaled.items():

        if osmolyte == "Other":
            for other in otherlist:
                try:
                    constraints_open = [model.model.problem.Constraint((model.model.reactions.get_by_id(other + "_v_gc_Linker_" + str(phase)).flux_expression+model.model.reactions.get_by_id(other + "_c_gc_Linker_" + str(phase)).flux_expression)/model.getvolumes(perguardcell = False)[phase-1],
                        lb=concentration[1][0],
                        ub=concentration[1][1]) for phase in [2]]
                    constraints_closed = [model.model.problem.Constraint((model.model.reactions.get_by_id(other + "_v_gc_Linker_" + str(phase)).flux_expression+model.model.reactions.get_by_id(other + "_c_gc_Linker_" + str(phase)).flux_expression)/model.getvolumes(perguardcell = False)[phase-1],
                        lb=concentration[0][0],
                        ub=concentration[0][1]) for phase in [1,3,4]]
                except:
                    constraints_open = [model.model.problem.Constraint((model.model.reactions.get_by_id(other + "_v_gc_Linker_" + str(phase)).flux_expression)/model.getvolumes(perguardcell = False)[phase-1],
                        lb=concentration[1][0],
                        ub=concentration[1][1]) for phase in [2]]
                    constraints_closed = [model.model.problem.Constraint((model.model.reactions.get_by_id(other + "_v_gc_Linker_" + str(phase)).flux_expression)/model.getvolumes(perguardcell = False)[phase-1],
                        lb=concentration[0][0],
                        ub=concentration[0][1]) for phase in [1,3,4]]
                for constraint in constraints_open+constraints_closed:
                    model.model.add_cons_vars(constraint)

        elif osmolyte == "MAL":
            constraints_open = [model.model.problem.Constraint((model.model.reactions.get_by_id(osmolyte + "_v_gc_Linker_" + str(phase)).flux_expression+model.model.reactions.get_by_id("a" + osmolyte + "_v_gc_Linker_" + str(phase)).flux_expression+model.model.reactions.get_by_id(osmolyte + "_c_gc_Linker_" + str(phase)).flux_expression)/model.getvolumes(perguardcell = False)[phase-1],
                        lb=concentration[1][0],
                        ub=concentration[1][1]) for phase in [2]]
            constraints_closed = [model.model.problem.Constraint((model.model.reactions.get_by_id(osmolyte + "_v_gc_Linker_" + str(phase)).flux_expression+model.model.reactions.get_by_id("a" + osmolyte + "_v_gc_Linker_" + str(phase)).flux_expression+model.model.reactions.get_by_id(osmolyte + "_c_gc_Linker_" + str(phase)).flux_expression)/model.getvolumes(perguardcell = False)[phase-1],
                lb=concentration[0][0],
                ub=concentration[0][1]) for phase in [1,3,4]]
            for constraint in constraints_open+constraints_closed:
                model.model.add_cons_vars(constraint)

        else:
            constraints_open = [model.model.problem.Constraint((model.model.reactions.get_by_id(osmolyte + "_v_gc_Linker_" + str(phase)).flux_expression+model.model.reactions.get_by_id(osmolyte + "_c_gc_Linker_" + str(phase)).flux_expression)/model.getvolumes(perguardcell = False)[phase-1],
                        lb=concentration[1][0],
                        ub=concentration[1][1]) for phase in [2]]
            constraints_closed = [model.model.problem.Constraint((model.model.reactions.get_by_id(osmolyte + "_v_gc_Linker_" + str(phase)).flux_expression+model.model.reactions.get_by_id(osmolyte + "_c_gc_Linker_" + str(phase)).flux_expression)/model.getvolumes(perguardcell = False)[phase-1],
                lb=concentration[0][0],
                ub=concentration[0][1]) for phase in [1,3,4]]
            for constraint in constraints_open+constraints_closed:
                model.model.add_cons_vars(constraint)